# Machine Learning Engineer Nanodegree
## Supervised Learning
## Project: Building a Student Intervention System

Welcome to the second project of the Machine Learning Engineer Nanodegree! In this notebook, some template code has already been provided for you, and it will be your job to implement the additional functionality necessary to successfully complete this project. Sections that begin with **'Implementation'** in the header indicate that the following block of code will require additional functionality which you must provide. Instructions will be provided for each section and the specifics of the implementation are marked in the code block with a `'TODO'` statement. Please be sure to read the instructions carefully!

In addition to implementing code, there will be questions that you must answer which relate to the project and your implementation. Each section where you will answer a question is preceded by a **'Question X'** header. Carefully read each question and provide thorough answers in the following text boxes that begin with **'Answer:'**. Your project submission will be evaluated based on your answers to each of the questions and the implementation you provide.  

>**Note:** Code and Markdown cells can be executed using the **Shift + Enter** keyboard shortcut. In addition, Markdown cells can be edited by typically double-clicking the cell to enter edit mode.

### Question 1 - Classification vs. Regression
*Your goal for this project is to identify students who might need early intervention before they fail to graduate. Which type of supervised learning problem is this, classification or regression? Why?*

**Answer: **

This is a classification problem. The output we are trying to find are discrete labels for whether a student will pass on not. (Passed = yes/no). For regression, the output is continuous/number.




## Exploring the Data
Run the code cell below to load necessary Python libraries and load the student data. Note that the last column from this dataset, `'passed'`, will be our target label (whether the student graduated or didn't graduate). All other columns are features about each student.

In [65]:
# Import libraries
import numpy as np
import pandas as pd
from time import time
from sklearn.metrics import f1_score

# Read student data
student_data = pd.read_csv("student-data.csv")
print "Student data read successfully!"


Student data read successfully!


#### Implementation: Data Exploration
Let's begin by investigating the dataset to determine how many students we have information on, and learn about the graduation rate among these students. In the code cell below, you will need to compute the following:
- The total number of students, `n_students`.
- The total number of features for each student, `n_features`.
- The number of those students who passed, `n_passed`.
- The number of those students who failed, `n_failed`.
- The graduation rate of the class, `grad_rate`, in percent (%).


In [66]:
# TODO: Calculate number of students
n_students = len(student_data.index)

# TODO: Calculate number of features
n_features = len(student_data.columns) - 1
# -1 removes the passed/failed column from the count

# TODO: Calculate passing students
n_passed = len(student_data[student_data.passed == 'yes'])

# TODO: Calculate failing students
n_failed = len(student_data[student_data.passed == 'no'])

# TODO: Calculate graduation rate
grad_rate = float(n_passed)*100/n_students

# Print the results
print "Total number of students: {}".format(n_students)
print "Number of features: {}".format(n_features)
print "Number of students who passed: {}".format(n_passed)
print "Number of students who failed: {}".format(n_failed)
print "Graduation rate of the class: {:.2f}%".format(grad_rate)


Total number of students: 395
Number of features: 30
Number of students who passed: 265
Number of students who failed: 130
Graduation rate of the class: 67.09%


## Preparing the Data
In this section, we will prepare the data for modeling, training and testing.

### Identify feature and target columns
It is often the case that the data you obtain contains non-numeric features. This can be a problem, as most machine learning algorithms expect numeric data to perform computations with.

Run the code cell below to separate the student data into feature and target columns to see if any features are non-numeric.


In [67]:
# Extract feature columns
feature_cols = list(student_data.columns[:-1])

# Extract target column 'passed'
target_col = student_data.columns[-1] 

# Show the list of columns
print "Feature columns:\n{}".format(feature_cols)
print "\nTarget column: {}".format(target_col)

# Separate the data into feature data and target data (X_all and y_all, respectively)
X_all = student_data[feature_cols]
y_all = student_data[target_col]

# Show the feature information by printing the first five rows
print "\nFeature values:"
print X_all.head()


Feature columns:
['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

Target column: passed

Feature values:
  school sex  age address famsize Pstatus  Medu  Fedu     Mjob      Fjob  \
0     GP   F   18       U     GT3       A     4     4  at_home   teacher   
1     GP   F   17       U     GT3       T     1     1  at_home     other   
2     GP   F   15       U     LE3       T     1     1  at_home     other   
3     GP   F   15       U     GT3       T     4     2   health  services   
4     GP   F   16       U     GT3       T     3     3    other     other   

    ...    higher internet  romantic  famrel  freetime goout Dalc Walc health  \
0   ...       yes       no        no       4         3     4    1    1      3   
1   ...       

### Preprocess Feature Columns

As you can see, there are several non-numeric columns that need to be converted! Many of them are simply `yes`/`no`, e.g. `internet`. These can be reasonably converted into `1`/`0` (binary) values.

Other columns, like `Mjob` and `Fjob`, have more than two values, and are known as _categorical variables_. The recommended way to handle such a column is to create as many columns as possible values (e.g. `Fjob_teacher`, `Fjob_other`, `Fjob_services`, etc.), and assign a `1` to one of them and `0` to all others.

These generated columns are sometimes called _dummy variables_, and we will use the [`pandas.get_dummies()`](http://pandas.pydata.org/pandas-docs/stable/generated/pandas.get_dummies.html?highlight=get_dummies#pandas.get_dummies) function to perform this transformation. Run the code cell below to perform the preprocessing routine discussed in this section.

In [68]:
def preprocess_features(X):
    ''' Preprocesses the student data and converts non-numeric binary variables into
        binary (0/1) variables. Converts categorical variables into dummy variables. '''
    
    # Initialize new output DataFrame
    output = pd.DataFrame(index = X.index)

    # Investigate each feature column for the data
    for col, col_data in X.iteritems():
        
        # If data type is non-numeric, replace all yes/no values with 1/0
        if col_data.dtype == object:
            col_data = col_data.replace(['yes', 'no'], [1, 0])

        # If data type is categorical, convert to dummy variables
        if col_data.dtype == object:
            # Example: 'school' => 'school_GP' and 'school_MS'
            col_data = pd.get_dummies(col_data, prefix = col)  
        
        # Collect the revised columns
        output = output.join(col_data)
    
    return output

X_all = preprocess_features(X_all)
print "Processed feature columns ({} total features):\n{}".format(len(X_all.columns), list(X_all.columns))


Processed feature columns (48 total features):
['school_GP', 'school_MS', 'sex_F', 'sex_M', 'age', 'address_R', 'address_U', 'famsize_GT3', 'famsize_LE3', 'Pstatus_A', 'Pstatus_T', 'Medu', 'Fedu', 'Mjob_at_home', 'Mjob_health', 'Mjob_other', 'Mjob_services', 'Mjob_teacher', 'Fjob_at_home', 'Fjob_health', 'Fjob_other', 'Fjob_services', 'Fjob_teacher', 'reason_course', 'reason_home', 'reason_other', 'reason_reputation', 'guardian_father', 'guardian_mother', 'guardian_other', 'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']


### Implementation: Training and Testing Data Split
So far, we have converted all _categorical_ features into numeric values. For the next step, we split the data (both features and corresponding labels) into training and test sets. In the following code cell below, you will need to implement the following:
- Randomly shuffle and split the data (`X_all`, `y_all`) into training and testing subsets.
  - Use 300 training points (approximately 75%) and 95 testing points (approximately 25%).
  - Set a `random_state` for the function(s) you use, if provided.
  - Store the results in `X_train`, `X_test`, `y_train`, and `y_test`.

In [69]:
# TODO: Import any additional functionality you may need here
from sklearn.model_selection import train_test_split

# TODO: Set the number of training points
num_train = 300

# Set the number of testing points
num_test = X_all.shape[0] - num_train

# TODO: Shuffle and split the dataset into the number of training and testing points above

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, train_size=num_train, random_state=0)

#X_train = None
#X_test = None
#y_train = None
#y_test = None

# Show the results of the split
print "Training set has {} samples.".format(X_train.shape[0])
print "Testing set has {} samples.".format(X_test.shape[0])

Training set has 300 samples.
Testing set has 95 samples.


## Training and Evaluating Models
In this section, you will choose 3 supervised learning models that are appropriate for this problem and available in `scikit-learn`. You will first discuss the reasoning behind choosing these three models by considering what you know about the data and each model's strengths and weaknesses. You will then fit the model to varying sizes of training data (100 data points, 200 data points, and 300 data points) and measure the F<sub>1</sub> score. You will need to produce three tables (one for each model) that shows the training set size, training time, prediction time, F<sub>1</sub> score on the training set, and F<sub>1</sub> score on the testing set.

**The following supervised learning models are currently available in** [`scikit-learn`](http://scikit-learn.org/stable/supervised_learning.html) **that you may choose from:**
- Gaussian Naive Bayes (GaussianNB)
- Decision Trees
- Ensemble Methods (Bagging, AdaBoost, Random Forest, Gradient Boosting)
- K-Nearest Neighbors (KNeighbors)
- Stochastic Gradient Descent (SGDC)
- Support Vector Machines (SVM)
- Logistic Regression

### Question 2 - Model Application
*List three supervised learning models that are appropriate for this problem. For each model chosen*
- Describe one real-world application in industry where the model can be applied. *(You may need to do a small bit of research for this — give references!)* 
- What are the strengths of the model; when does it perform well? 
- What are the weaknesses of the model; when does it perform poorly?
- What makes this model a good candidate for the problem, given what you know about the data?

** Answer: **

I tried SVMs and Random Forests as they are considered high performance algorithms. I also added decision trees.
http://blog.echen.me/2011/04/27/choosing-a-machine-learning-classifier/


** Decision Trees **

Decision trees can be used for a wide range of applications. In this example, a decision tree is used for identifying if 2 records from a census are the same person. https://digital.library.txstate.edu/handle/10877/5576

Here is another where decision trees were used to identify patients with severe dengue infection. http://journals.plos.org/plosone/article?id=10.1371/journal.pone.0161696

* Strengths: Graphically, they allow you to interpret the data really well. They can be easy to explain. They are very fast.


* Weakness: 
Prone to overfitting. In data with a lot of features, decision trees can overfit and we have to be careful with parameter tunes to avoid this. Decision trees can be used to build additional classifiers with ensemble methods.


* Appropriateness to data set:
This data set is fairly small and has relatively few features. 


** Support Vector Machines **

Support Vector Machines can be used for classification tasks like text classification and image recognition. Handwritten number recognition, face recognition etc are examples called out in the papers below.

http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.723.5893&rep=rep1&type=pdf
https://pdfs.semanticscholar.org/62d9/d8b5143348afdb3b96f18527a424b2a6260f.pdf

* Strengths: 
SVMs work well in domains with a clear margin of separation. SVM has a regularization parameter C that can be tuned to avoid overfitting. Kernel functions allow you to encode expert knowledge or prior knowledge of problems related to a specific domain into the SVM - http://mrebl.web.engr.illinois.edu/Sun_05a.pdf


* Weaknesses:
They don't work well in large data sets because the training time tends to be cubic the size of the data set. For large datasets, SVMs could be slow. And they can be prone to overfitting to the noise in the data.


* Appropriateness to data set:
This data set is fairly small and has relatively few features and we can test out SVM. Number of features is lower than the number of observations in the dataset and a SVM may perform well. Given SVMs tend to be accurate classifiers, I wanted to try it on this set. 


** Random Forest **

Ensemble methods including random forests are widely used as they tend to be accurate classifiers. Here are 2 real-world examples.

Diagnosis of Chronic Kidney Disease by Using Random Forest
https://link.springer.com/chapter/10.1007/978-981-10-4166-2_89

Reading CT images of vertebrae:
https://books.google.com/books?hl=en&lr=&id=3iA9DgAAQBAJ&oi=fnd&pg=PA51&dq=random+forest&ots=OQp7mShxr7&sig=h6ExAo2hR4mQN1A6iqdgKTRAMng#v=onepage&q=random%20forest&f=false

* Strengths:
They are widely used in industry and tend to be one of the more accurate classifiers. They can deal well with missing data and with data where the classes are unbalanced.


* Weaknesses:
Not easy to understand how the model was trained or to explain the model - unlike a decision tree. Random forests can be fast to train, but slow to predict, since accurate results may require greater number of trees. So, applications where run-time speed is important, another algorithm may be better.


* Applicability to current data set:
Data set/features is small enough that it will not be computationally too expensive to try random forests. Given the wide use in industry applications and the fact that they are considered accurate classifiers, it is useful to try it on this data set.


In addition to the 3 models mentioned above, I also try running a Gaussian Naive Bayes model in the code below. 





### Setup
Run the code cell below to initialize three helper functions which you can use for training and testing the three supervised learning models you've chosen above. The functions are as follows:
- `train_classifier` - takes as input a classifier and training data and fits the classifier to the data.
- `predict_labels` - takes as input a fit classifier, features, and a target labeling and makes predictions using the F<sub>1</sub> score.
- `train_predict` - takes as input a classifier, and the training and testing data, and performs `train_clasifier` and `predict_labels`.
 - This function will report the F<sub>1</sub> score for both the training and testing data separately.

In [70]:
def train_classifier(clf, X_train, y_train):
    ''' Fits a classifier to the training data. '''
    
    # Start the clock, train the classifier, then stop the clock
    start = time()
    clf.fit(X_train, y_train)
    end = time()
    
    # Print the results
    print "Trained model in {:.4f} seconds".format(end - start)

    
def predict_labels(clf, features, target):
    ''' Makes predictions using a fit classifier based on F1 score. '''
    
    # Start the clock, make predictions, then stop the clock
    start = time()
    y_pred = clf.predict(features)
    end = time()
    
    # Print and return results
    print "Made predictions in {:.4f} seconds.".format(end - start)
    return f1_score(target.values, y_pred, pos_label='yes')


def train_predict(clf, X_train, y_train, X_test, y_test):
    ''' Train and predict using a classifer based on F1 score. '''
    
    # Indicate the classifier and the training set size
    print "Training a {} using a training set size of {}. . .".format(clf.__class__.__name__, len(X_train))
    
    # Train the classifier
    train_classifier(clf, X_train, y_train)
    
    # Print the results of prediction for both training and testing
    print "F1 score for training set: {:.4f}.".format(predict_labels(clf, X_train, y_train))
    print "F1 score for test set: {:.4f}.".format(predict_labels(clf, X_test, y_test))
    


### Implementation: Model Performance Metrics
With the predefined functions above, you will now import the three supervised learning models of your choice and run the `train_predict` function for each one. Remember that you will need to train and predict on each classifier for three different training set sizes: 100, 200, and 300. Hence, you should expect to have 9 different outputs below — 3 for each model using the varying training set sizes. In the following code cell, you will need to implement the following:
- Import the three supervised learning models you've discussed in the previous section.
- Initialize the three models and store them in `clf_A`, `clf_B`, and `clf_C`.
 - Use a `random_state` for each model you use, if provided.
 - **Note:** Use the default settings for each model — you will tune one specific model in a later section.
- Create the different training set sizes to be used to train each model.
 - *Do not reshuffle and resplit the data! The new training points should be drawn from `X_train` and `y_train`.*
- Fit each model with each training set size and make predictions on the test set (9 in total).  
**Note:** Three tables are provided after the following code cell which can be used to store your results.

In [71]:
# TODO: Import the three supervised learning models from sklearn

from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

# from sklearn import model_B
# from sklearn import model_C

# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=0)
clf_B = SVC(random_state=0)
clf_C = RandomForestClassifier(n_estimators=10, random_state=0)
clf_D = GaussianNB()

# TODO: Set up the training set sizes
X_train_100 = X_train[:100]
y_train_100 = y_train[:100]

X_train_200 = X_train[:200]
y_train_200 = y_train[:200]

X_train_300 = X_train[:300]
y_train_300 = y_train[:300]


# TODO: Execute the 'train_predict' function for each classifier and each training set size

for clf in [clf_A, clf_B, clf_C, clf_D]:
    for n in [100, 200, 300]:
        train_predict(clf, X_train[:n], y_train[:n], X_test, y_test)
        print ""
        
#Shows the importance of various features

pd.DataFrame(zip(X_train.columns, clf_C.feature_importances_),columns=['feature', 'importance']).sort('importance', ascending = False)

Training a DecisionTreeClassifier using a training set size of 100. . .
Trained model in 0.0019 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0006 seconds.
F1 score for test set: 0.6942.

Training a DecisionTreeClassifier using a training set size of 200. . .
Trained model in 0.0021 seconds
Made predictions in 0.0003 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0004 seconds.
F1 score for test set: 0.7132.

Training a DecisionTreeClassifier using a training set size of 300. . .
Trained model in 0.0042 seconds
Made predictions in 0.0005 seconds.
F1 score for training set: 1.0000.
Made predictions in 0.0003 seconds.
F1 score for test set: 0.7167.

Training a SVC using a training set size of 100. . .
Trained model in 0.0019 seconds
Made predictions in 0.0011 seconds.
F1 score for training set: 0.8591.
Made predictions in 0.0009 seconds.
F1 score for test set: 0.7838.

Training a SVC using a training set size of 200

/Users/resmi/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:35: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)


,feature,importance
47,absences,0.080199
43,goout,0.071354
32,failures,0.054991
46,health,0.049152
11,Medu,0.047424
45,Walc,0.045249
41,famrel,0.041782
44,Dalc,0.032260
42,freetime,0.031973
12,Fedu,0.031896


### Tabular Results
Edit the cell below to see how a table can be designed in [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet#tables). You can record your results from above in the tables provided.

The top algorithms (based on F1 score on test set above are included below)

** Classifer 1 - Decision Tree **

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |  0.0011                 |          .0002         |     1            |  0.6942         |
| 200               |  0.0014                 |       .0002            |      1           |  0.7132         |
| 300               |     0.0019                    |       .0003                 |   1               |    0.7167    |

** Classifer 2 - Support Vector Machines **

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |   0.0013                |    0.0008              |  0.8591           |  0.7838        |
| 200               |     0.0034              |   0.0013               |  0.8693           | 0.7755         |
| 300               |     0.0065              |    0.0018              |  0.8692           |   0.7586       |

** Classifer 3 - Random Forest**  

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 100               |       0.0240            |     .0054              |     0.9841       |       0.7119    |
| 200               |          0.0247         |         0.0066         |        1         |    0.7761       |
| 300               |            0.0367       |         0.0053         |        0.9976    |        0.7344   |

** Classifer 4 - Gaussian Naive Bayes**  (including just the 300 points training set)

| Training Set Size | Training Time | Prediction Time (test) | F1 Score (train) | F1 Score (test) |
| :---------------: | :---------------------: | :--------------------: | :--------------: | :-------------: |
| 300               |            0.0024      |         0.0006         |        0.8088    |        0.7500   |





## Choosing the Best Model
In this final section, you will choose from the three supervised learning models the *best* model to use on the student data. You will then perform a grid search optimization for the model over the entire training set (`X_train` and `y_train`) by tuning at least one parameter to improve upon the untuned model's F<sub>1</sub> score. 

### Question 3 - Choosing the Best Model
*Based on the experiments you performed earlier, in one to two paragraphs, explain to the board of supervisors what single model you chose as the best model. Which model is generally the most appropriate based on the available data, limited resources, cost, and performance?*

**Answer: **

Based on the running a few models, we have selected Support Vector Machines as the most appropriate model. 

Based on training with 300 data points, the training times and prediction times were as follows:

* Decision Trees: 0.0019 and 0.0003
* SVMs: 0.0065 and 0.0018
* Random Forest: 0.0367 and 0.0053

The F1 score for test data was

* Decision Trees: 0.7167
* SVMs: 0.7586
* Random Forest: 0.7344

Based on these, SVMs were the most accurate. And the training time was higher than Decision Trees, but not too high. Random Forests had a much higher training time and prediction time for lower accuracy. Given that this is not a real-time model, training time and prediction time can be relatively high if it results in a more accurate model. Also,the dataset is not huge, so the time/cost is not high even with a longer training/prediction time. Hence we chose an SVM.


### Question 4 - Model in Layman's Terms
*In one to two paragraphs, explain to the board of directors in layman's terms how the final model chosen is supposed to work. Be sure that you are describing the major qualities of the model, such as how the model is trained and how the model makes a prediction. Avoid using advanced mathematical or technical jargon, such as describing equations or discussing the algorithm implementation.*

**Answer: **

Support Vector Machines are a type of supervised learning classifier. For supervised learning, we take some data that is already classified/labeled and train the classifier on this data. Once trained, the classifier can be used to predict the classes/labels on some test data. By comparing the predicted labels on the test data, with the actual labels on the test data, we can get a sense of the accuracy of the classifier. Once trained and modified to an acceptable level of accuracy, the classifer can be used to predict the classes/labels for unclassified data.

SVMs, the specific type of model chosen here, works by mapping the data points as points in space, mapped so that the examples of the separate categories are divided by a clear gap that is as wide as possible. New examples are then mapped into that same space and predicted to belong to a category based on which side of the gap they fall.

In addition to performing linear classification, SVMs can efficiently perform a non-linear classification using what is called the kernel trick, implicitly mapping their inputs into high-dimensional feature spaces where linear classification becomes possible.

https://en.wikipedia.org/wiki/Support_vector_machine


### Implementation: Model Tuning
Fine tune the chosen model. Use grid search (`GridSearchCV`) with at least one important parameter tuned with at least 3 different values. You will need to use the entire training set for this. In the code cell below, you will need to implement the following:
- Import [`sklearn.grid_search.GridSearchCV`](http://scikit-learn.org/0.17/modules/generated/sklearn.grid_search.GridSearchCV.html) and [`sklearn.metrics.make_scorer`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.make_scorer.html).
- Create a dictionary of parameters you wish to tune for the chosen model.
 - Example: `parameters = {'parameter' : [list of values]}`.
- Initialize the classifier you've chosen and store it in `clf`.
- Create the F<sub>1</sub> scoring function using `make_scorer` and store it in `f1_scorer`.
 - Set the `pos_label` parameter to the correct value!
- Perform grid search on the classifier `clf` using `f1_scorer` as the scoring method, and store it in `grid_obj`.
- Fit the grid search object to the training data (`X_train`, `y_train`), and store it in `grid_obj`.

In [72]:
# TODO: Import 'GridSearchCV' and 'make_scorer'
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer


# Here I try to find the optimal parameters for the SVM and the Random Forest classifier. 
# The goal is to see if optimizing improves the accuracy of the Random Forest to be higher than SVM.


# Optimizing SVM.

# TODO: Create the parameters list you wish to tune


parameters = [{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']}]

# TODO: Initialize the classifier
clf = SVC(random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj = GridSearchCV(clf, parameters, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj = grid_obj.fit(X_train, y_train)

# Get the estimator
clf = grid_obj.best_estimator_

best_parameters = clf.get_params()
for param_name in sorted(best_parameters):
    print(param_name, best_parameters[param_name])

# Report the final F1 score for training and testing after parameter tuning
print "Tuned SVM model has a training F1 score of {:.4f}.".format(predict_labels(clf, X_train, y_train))
print "Tuned SVM model has a testing F1 score of {:.4f}.".format(predict_labels(clf, X_test, y_test))
print ""

# Optimizing Random Forest Classifier

# TODO: Create the parameters list you wish to tune



parameters1 = [{"max_depth": [1, 2, 3, 4]}]
          
# TODO: Initialize the classifier
clf1 = RandomForestClassifier(n_estimators=10, random_state=0)

# TODO: Make an f1 scoring function using 'make_scorer' 
f1_scorer = make_scorer(f1_score, pos_label="yes")

# TODO: Perform grid search on the classifier using the f1_scorer as the scoring method
grid_obj1 = GridSearchCV(clf1, parameters1, scoring=f1_scorer)

# TODO: Fit the grid search object to the training data and find the optimal parameters
grid_obj1 = grid_obj1.fit(X_train, y_train)

# Get the estimator
clf1 = grid_obj1.best_estimator_

best_parameters1 = clf1.get_params()
for param_name in sorted(best_parameters1):
    print(param_name, best_parameters1[param_name])

# Report the final F1 score for training and testing after parameter tuning
print "Tuned Random Forest model has a training F1 score of {:.4f}.".format(predict_labels(clf1, X_train, y_train))
print "Tuned Random Forest model has a testing F1 score of {:.4f}.".format(predict_labels(clf1, X_test, y_test))







('C', 10)
('cache_size', 200)
('class_weight', None)
('coef0', 0.0)
('decision_function_shape', None)
('degree', 3)
('gamma', 0.001)
('kernel', 'rbf')
('max_iter', -1)
('probability', False)
('random_state', 0)
('shrinking', True)
('tol', 0.001)
('verbose', False)
Made predictions in 0.0063 seconds.
Tuned SVM model has a training F1 score of 0.8434.
Made predictions in 0.0019 seconds.
Tuned SVM model has a testing F1 score of 0.7862.

('bootstrap', True)
('class_weight', None)
('criterion', 'gini')
('max_depth', 2)
('max_features', 'auto')
('max_leaf_nodes', None)
('min_impurity_split', 1e-07)
('min_samples_leaf', 1)
('min_samples_split', 2)
('min_weight_fraction_leaf', 0.0)
('n_estimators', 10)
('n_jobs', 1)
('oob_score', False)
('random_state', 0)
('verbose', 0)
('warm_start', False)
Made predictions in 0.0069 seconds.
Tuned Random Forest model has a training F1 score of 0.8207.
Made predictions in 0.0053 seconds.
Tuned Random Forest model has a testing F1 score of 0.7815.


### Question 5 - Final F<sub>1</sub> Score
*What is the final model's F<sub>1</sub> score for training and testing? How does that score compare to the untuned model?*



**Answer: **

The final training score = 0.8434 and the final testing score = 0.7862. This compares to a score of 0.8692 and 0.7586 respectively for the untrained model. So, the final testing score increased from 0.7586 to 0.7862. The tuned model has a rbf kernel, C of 10 and gamma of .001. 

The tuned Random Forest model had a lower F1 score.        




> **Note**: Once you have completed all of the code implementations and successfully answered each question above, you may finalize your work by exporting the iPython Notebook as an HTML document. You can do this by using the menu above and navigating to  
**File -> Download as -> HTML (.html)**. Include the finished document along with this notebook as your submission.